<a href="https://colab.research.google.com/github/Starun26babu/Artifitial-Intelligence-and-Machine-Learning/blob/master/SeqNLP_Project1_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Classification


## Loading the dataset

In [0]:
import keras
import keras.utils
from keras import utils as np_utils
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backend
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

from keras.models import Sequential
from keras.layers import Dense
from keras.layers.embeddings import Embedding
from keras.layers import Flatten


vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

## Train test split

In [0]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [16]:
print ("x_train shape: ", x_train.shape)
print ("y_train shape: ", y_train.shape)
print ("x_test shape: ", x_test.shape)
print ("y_test shape: ", y_test.shape)

('x_train shape: ', (25000, 300))
('y_train shape: ', (25000,))
('x_test shape: ', (25000, 300))
('y_test shape: ', (25000,))


In [17]:
print("Maximum value of a word index ")
print(max([max(sequence) for sequence in x_train]))
print("Maximum length num words of review in train ")
print(max([len(sequence) for sequence in x_train]))

Maximum value of a word index 
9999
Maximum length num words of review in train 
300


In [18]:

x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [20]:

# See an actual review in words
# Reverse from integers to words using the DICTIONARY (given by keras...need to do nothing to create it)

word_index = imdb.get_word_index()

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_train[123]])
#reference from https://mc.ai/python-deep-learning-part-3/
print(decoded_review)
len(word_index)

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? beautiful and touching movie rich colors great settings good acting and one of the most charming movies i have seen in a while i never saw such an interesting setting when i was in china my wife liked it so much she asked me to ? on and rate it so other would enjoy too


88584

In [21]:
x_train[123]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [22]:

reverse_word_index

{1: u'the',
 2: u'and',
 3: u'a',
 4: u'of',
 5: u'to',
 6: u'is',
 7: u'br',
 8: u'in',
 9: u'it',
 10: u'i',
 11: u'this',
 12: u'that',
 13: u'was',
 14: u'as',
 15: u'for',
 16: u'with',
 17: u'movie',
 18: u'but',
 19: u'film',
 20: u'on',
 21: u'not',
 22: u'you',
 23: u'are',
 24: u'his',
 25: u'have',
 26: u'he',
 27: u'be',
 28: u'one',
 29: u'all',
 30: u'at',
 31: u'by',
 32: u'an',
 33: u'they',
 34: u'who',
 35: u'so',
 36: u'from',
 37: u'like',
 38: u'her',
 39: u'or',
 40: u'just',
 41: u'about',
 42: u"it's",
 43: u'out',
 44: u'has',
 45: u'if',
 46: u'some',
 47: u'there',
 48: u'what',
 49: u'good',
 50: u'more',
 51: u'when',
 52: u'very',
 53: u'up',
 54: u'no',
 55: u'time',
 56: u'she',
 57: u'even',
 58: u'my',
 59: u'would',
 60: u'which',
 61: u'only',
 62: u'story',
 63: u'really',
 64: u'see',
 65: u'their',
 66: u'had',
 67: u'can',
 68: u'were',
 69: u'me',
 70: u'well',
 71: u'than',
 72: u'we',
 73: u'much',
 74: u'been',
 75: u'bad',
 76: u'get',
 77: 

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [24]:


import os
# create the model
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=maxlen))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 32)           320000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               2400250   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 2,720,501
Trainable params: 2,720,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# Fit the model
output= model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/5
 - 14s - loss: 0.4358 - accuracy: 0.7737 - val_loss: 0.2889 - val_accuracy: 0.8783
Epoch 2/5
 - 14s - loss: 0.1325 - accuracy: 0.9547 - val_loss: 0.3455 - val_accuracy: 0.8638
Epoch 3/5
 - 14s - loss: 0.0245 - accuracy: 0.9947 - val_loss: 0.4538 - val_accuracy: 0.8584
Epoch 4/5
 - 14s - loss: 0.0036 - accuracy: 0.9997 - val_loss: 0.5064 - val_accuracy: 0.8611
Epoch 5/5
 - 14s - loss: 8.8028e-04 - accuracy: 1.0000 - val_loss: 0.5365 - val_accuracy: 0.8630
Accuracy: 86.30%


In [26]:

output.history

{'accuracy': [0.77368, 0.95468, 0.99472, 0.99972, 1.0],
 'loss': [0.43579367578506467,
  0.1324728762817383,
  0.024464459960460663,
  0.003605340094566345,
  0.0008802750789746642],
 'val_accuracy': [0.8783199787139893,
  0.8638399839401245,
  0.8583599925041199,
  0.8611199855804443,
  0.8629599809646606],
 'val_loss': [0.2888761562347412,
  0.34545636776447297,
  0.45377384940147397,
  0.506364543542862,
  0.5364708729171753]}

## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [0]:
BATCH_SIZE=128
EPOCHS=4

In [28]:
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.1,
          verbose=2)

Train on 22500 samples, validate on 2500 samples
Epoch 1/4
 - 11s - loss: 4.5825e-04 - accuracy: 1.0000 - val_loss: 4.2062e-04 - val_accuracy: 1.0000
Epoch 2/4
 - 11s - loss: 3.0841e-04 - accuracy: 1.0000 - val_loss: 3.3048e-04 - val_accuracy: 1.0000
Epoch 3/4
 - 11s - loss: 2.2279e-04 - accuracy: 1.0000 - val_loss: 2.8028e-04 - val_accuracy: 1.0000
Epoch 4/4
 - 11s - loss: 1.6818e-04 - accuracy: 1.0000 - val_loss: 2.4180e-04 - val_accuracy: 1.0000


In [29]:
# Evaluate the model
score, acc = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("Accuracy: %.2f%%" % (acc*100))

pred = model.predict_classes(x_test)

25000/25000 [==============================] - 2s 87us/step
Accuracy: 86.54%


In [30]:
from sklearn.metrics import confusion_matrix
LABELS = ['negative', 'positive']


# Print Precision Recall F1-Score Report
from sklearn.metrics import classification_report

report = classification_report(pred, y_test, target_names=LABELS)
print(report)

              precision    recall  f1-score   support

    negative       0.87      0.86      0.87     12528
    positive       0.86      0.87      0.87     12472

   micro avg       0.87      0.87      0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
clf = MultinomialNB()
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
print(accuracy_score(y_test, pred))

0.50016
